In [323]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/piworksbus/municipality_bus_utilization.csv


In [324]:
#import libraries
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [325]:
#upload the dataset
url= '/kaggle/input/piworksbus/municipality_bus_utilization.csv'
df= pd.read_csv(url,sep=',')

#Deal with missing values using interpolation
df.interpolate(method='linear', axis=0, limit=None, inplace=True)

#Put in date-time format
df['timestamp'] = pd.to_datetime(df['timestamp'])

#set the date as index
df.set_index('timestamp',drop=False,inplace=True)

#resample the data hourly regarding to the max value
hourly_data = df.resample('H').max()

# Add lag features, as baseline simple solution we cad only one lag factor which is usage_LAG_-1 to be the target
for i in range(-1, 5):
    hourly_data[f'usage_LAG_{i}'] = hourly_data['usage'].shift(i)

hourly_data.dropna(inplace=True)
hourly_data

,timestamp,municipality_id,usage,total_capacity,usage_LAG_-1,usage_LAG_0,usage_LAG_1,usage_LAG_2,usage_LAG_3,usage_LAG_4
timestamp,,,,,,,,,,
2017-06-04 11:00:00,2017-06-04 11:59:44,9.0,3257.0,3893.0,3260.0,3257.0,3178.0,2811.0,2016.0,1090.0
2017-06-04 12:00:00,2017-06-04 12:29:45,9.0,3260.0,3893.0,3241.0,3260.0,3257.0,3178.0,2811.0,2016.0
2017-06-04 13:00:00,2017-06-04 13:29:45,9.0,3241.0,3893.0,3154.0,3241.0,3260.0,3257.0,3178.0,2811.0
2017-06-04 14:00:00,2017-06-04 14:57:13,9.0,3154.0,3893.0,2848.0,3154.0,3241.0,3260.0,3257.0,3178.0
2017-06-04 15:00:00,2017-06-04 15:30:14,9.0,2848.0,3893.0,2665.0,2848.0,3154.0,3241.0,3260.0,3257.0
...,...,...,...,...,...,...,...,...,...,...
2017-08-18 15:00:00,2017-08-18 15:30:24,9.0,1764.0,3893.0,1647.0,1764.0,1893.0,1819.0,1723.0,1387.0
2017-08-19 12:00:00,2017-08-19 12:30:32,9.0,3157.0,3893.0,3194.0,3157.0,2986.0,2785.0,2613.0,1724.0
2017-08-19 13:00:00,2017-08-19 13:30:35,9.0,3194.0,3893.0,3183.0,3194.0,3157.0,2986.0,2785.0,2613.0


In [326]:
# Split data into training and testing sets
train_start_date = '2017-06-04 00:00:00'
train_end_date = '2017-08-04 23:00:00'
test_start_date = '2017-08-05 00:00:00'
test_end_date = '2017-08-19 23:00:00'
last_week_start_date= '2017-08-13 00:00:00'

train_data = hourly_data.loc[(hourly_data['timestamp'] >= train_start_date) & (hourly_data['timestamp'] <= train_end_date)]
test_data = hourly_data.loc[(hourly_data['timestamp'] >= test_start_date) & (hourly_data['timestamp'] <= test_end_date)]
#prepare data of only last week for the main task
last_week_data= hourly_data.loc[(hourly_data['timestamp'] >= last_week_start_date) & (hourly_data['timestamp'] <= test_end_date)]


# Define features and target
X_train = train_data.drop(['usage_LAG_0','usage_LAG_-1','timestamp'], axis=1).values 
y_train = (train_data['usage_LAG_-1']).values
X_test = test_data.drop([ 'usage_LAG_0','usage_LAG_-1','timestamp'], axis=1).values
y_test = (test_data['usage_LAG_-1']).values
X_last_week = last_week_data.drop([ 'usage_LAG_0','usage_LAG_-1','timestamp'], axis=1).values


In [327]:
#common function to evaluate the models
def evaluate(y_test,y_pred):
    #calculate the R^2 value
    r2 = r2_score(y_test, y_pred)*100
    #calculate the MAPE value
    mape= mean_absolute_percentage_error(y_test,y_pred)*100
    
    #print the results
    print(f"The R\u00b2 value is: {r2:0.4f}%")
    print(f"The MAPE value is: {mape:0.4f}%")
    


In [328]:
# Fit and evaluate linear regression model as simple baseline method
lr = LinearRegression()
lr.fit(X_train, y_train)

#predict and evaluate the test data
y_pred_lr = lr.predict(X_test)

print('Linear regression model evaluation')
evaluate(y_test,y_pred_lr)

#Forecast hourly data for the next week
y_forecasted_week_lr = lr.predict(X_last_week)
print(f"Linear regression model forecasted week usage:{np.round(y_forecasted_week_lr, 1)}")

Linear regression model evaluation
The R² value is: 96.7478%
The MAPE value is: 3.4116%
Linear regression model forecasted week usage:[3602.7 3320.  3119.1 2745.8 3302.  3285.  3124.5 2620.4 3320.4 3401.7
 3271.6 3184.7 2907.7 2999.5 2973.3 2839.8 2543.3 2170.7 1693.6 1873.8
 1976.9 2171.1 2150.5 1887.1 1857.6 1836.6 1625.4 3293.4 3083.1 3014.
 2893.1]


In [329]:
# Fit and evaluate linear regression model as simple baseline method
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
gb.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb.predict(X_test)

print('Gradient boosting model evaluation')
evaluate(y_test,y_pred_gb)

#Forecast hourly data for the next week
y_forecasted_week_gb = gb.predict(X_last_week)
print(f"Gradient boosting model forecasted week usage:{np.round(y_forecasted_week_gb, 1)}")

Gradient boosting model evaluation
The R² value is: 97.0967%
The MAPE value is: 2.8612%
Gradient boosting model forecasted week usage:[3475.2 3429.6 3136.6 2725.8 3369.4 3320.8 3132.1 2550.7 3330.  3364.8
 3303.  3248.  2870.3 2959.4 2972.1 2782.1 2604.3 2150.5 1660.2 1751.4
 1751.6 2118.6 2050.2 1819.6 1855.2 1720.4 1646.8 3122.9 3141.3 3113.3
 3006. ]
